# Images

## Reading

In [2]:
import os
import zipfile
import numpy as np
import tensorflow as tf  # for data preprocessing
import nibabel as nib
from scipy import ndimage

import keras
from keras import layers
# tf.enable_eager_execution()

In [3]:
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    try:
        scan = nib.load(filepath)
        # Get raw data
        scan = scan.get_fdata()
        return scan
    except Exception as e:
        print(e)
        return None

scan_paths = [
    os.path.join(os.getcwd(), "../image_slicing/sample", x)
    for x in os.listdir("../image_slicing/sample")
]

print("CT scans with normal lung tissue: " + str(len(scan_paths)))

CT scans with normal lung tissue: 4


In [4]:
# Scan masks to sort between positive and negative samples
mask_scans_list = [read_nifti_file(path + '/MASK.nii.gz') for path in scan_paths]
ok_idx_mask = [i for i, x in enumerate(mask_scans_list) if x is not None]
ct_scans_list = [read_nifti_file(path + '/CT.nii.gz') for path in scan_paths]
ok_idx_ct = [i for i, x in enumerate(ct_scans_list) if x is not None]
pet_scans_list = [read_nifti_file(path + '/PET.nii.gz') for path in scan_paths]
ok_idx_pet = [i for i, x in enumerate(pet_scans_list) if x is not None]

# Filter for corrupted files
ok_indices = set(ok_idx_mask) & set(ok_idx_ct) & set(ok_idx_pet)
print(ok_indices)

ct_scans_list = [ct_scans_list[i] for i in ok_indices]
mask_scans_list = [mask_scans_list[i] for i in ok_indices]
pet_scans_list = [pet_scans_list[i] for i in ok_indices]

{0, 1, 2, 3}


In [5]:
# find positive and negative samples: if the mask has any pixel > 0 it is a positive sample
pos_indices = np.where([np.max(mask) > 0 for mask in mask_scans_list])[0]
del mask_scans_list

## Processing

In [6]:
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 250
    desired_width = 350
    desired_height = 350
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    #img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    img = img.astype(np.float16)
    return img

def process_scan(scan):
    """Read and resize volume"""
    # Normalize
    volume = normalize(scan)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume


In [7]:
# mask_scans_processed = np.array([process_scan(scan) for scan in mask_scans_list])
ct_scans_processed = np.array([process_scan(scan) for scan in ct_scans_list])
combined_scans = 0.02 * ct_scans_processed
del ct_scans_processed
del ct_scans_list

pet_scans_processed = np.array([process_scan(scan) for scan in pet_scans_list])
combined_scans += 0.98 * pet_scans_processed
del pet_scans_processed
del pet_scans_list

In [8]:
import gc
gc.collect()

255

## Preparing

In [9]:
X = combined_scans[..., np.newaxis]  # Add channel dimension
y = np.array([1 if i in pos_indices else 0 for i in range(len(combined_scans))])  # Assuming pos_indices identifies positive samples
del combined_scans

# Split data (consider using sklearn's train_test_split)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense

model = Sequential([
    Conv3D(16, kernel_size=(3, 3, 3), activation='relu', input_shape=(350, 350, 250, 1)),  # Adjusted input shape
    MaxPooling3D(pool_size=(2, 2, 2)),
    Conv3D(32, kernel_size=(3, 3, 3), activation='relu'),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])


/Users/domen/mypython/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
# AFTER:
from tensorflow.keras.callbacks import ModelCheckpoint
import os

# Checkpoint directory
checkpoint_dir = "checkpoints/"

# Create directory if it does not exist
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

checkpoint_path = "checkpoints/model_{epoch:02d}-{val_loss:.2f}.keras"

# Create a ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_best_only=True,  # Save only the best model
                             monitor='val_loss',   # Monitor the validation loss
                             mode='min',           # Minimize the validation loss
                             verbose=1)            # Print out when a model is being saved


In [17]:
# Pass this callback to the `fit` method
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=1,
    verbose=1,
    batch_size=2,
    callbacks=[checkpoint]  # Include the callback here
)